# TwInSolar Summer School Satellite Exercises

Welcome to the TwInSolar Summer School Satellite Exercises! This notebook contains a set of exercises designed to help you explore and analyse satellite irradiance data.

### Prerequisites

Before you begin, please ensure that you have the necessary dependencies installed. This notebook requires the following Python libraries:
- xarray
- numpy
- matplotlib
- pandas
- jupyter

### Setting Up the Environment

To streamline the setup process, we recommend creating a dedicated Conda environment for this notebook. Follow the steps below to create and activate your environment, and install the required packages:

1. Create a new Conda environment: `conda create --name twinsolar_sumemrschool_env`

2. Activate the newly created environment: `conda activate twinsolar_sumemrschool_env`

3. Install the necessary packages: `conda install xarray matplotlib pandas numpy jupyter`

4. Launch Jupyter Notebook: `jupyter notebook`

Once Jupyter Notebook is running, open this notebook to begin the exercises.

### Data Provided

Included with this notebook is a NetCDF file containing irradiance data. We will use this file to perform various analyses and visualisations throughout the exercises.

### xarray

For those unfamiliar, xarray is a powerful and flexible Python library designed for working with labeled multi-dimensional arrays, which are commonly encountered in scientific data analysis. It extends the capabilities of NumPy arrays by adding labels to dimensions, coordinates, and attributes, making it easier to work with complex datasets like those found in climate science, oceanography, and remote sensing.

#### Key Features of xarray:
- **Labeled Dimensions and Coordinates:** Access and manipulate data using meaningful labels rather than just integer indices.
- **Dataset and DataArray Objects:** Organize multiple arrays into a single dataset, preserving the relationships between variables.
- **Integration with NetCDF:** Seamlessly read and write NetCDF files, a common format for storing scientific data.
- **Interoperability:** Works well with other scientific libraries such as NumPy, Pandas, and Matplotlib.
- **Efficient Computation:** Supports parallel computation and lazy evaluation to handle large datasets efficiently.

For more detailed information, please refer to the [xarray documentation](https://docs.xarray.dev/en/stable/getting-started-guide/index.html).

### Let's Get Started!

With your environment set up and the dependencies installed, you're ready to dive into the satellite data exercises.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import urllib.request

In [ ]:
# import nc file from GitHub
nc_file = "20220701_20230101_sat_irrad_forecasts.nc"
url = "https://github.com/Laboratoire-Piment/TwInSolar-Summer_shool_data/raw/main/"
urllib.request.urlretrieve(url+nc_file, nc_file)

In [3]:
# edit the path to the sat_dataset_path

sat_dataset_path = "20220701_20230101_sat_irrad_forecasts.nc"

ds = xr.open_dataset(sat_dataset_path)

In [ ]:
ds

## 0. Description of the NetCDF File

This NetCDF file contains irradiance data derived from satellite-based cloud motion vector irradiance forecasts. It includes multiple dimensions, coordinates, and data variables that provide comprehensive information about the irradiance measurements over time and space.

#### Dimensions:
- **base_time:** ~ 17k steps representing the base time points.
- **step:** 13 steps indicating the time steps from the base time.
- **location_id:** 1 location identifier, for one solar site in La Reunion

#### Coordinates:
- **base_time:** A datetime coordinate ranging from `2022-07-01T00:30:00` to `2023-01-01T00:00:00`, representing the base times of the measurements.
- **step:** A timedelta coordinate ranging from `00:00:00` to `06:00:00`, representing the time steps after the base time.
- **location_id:** An integer coordinate for location identification, always 0 since we only have one location.
- **lat:** Latitude values for the given location.
- **lon:** Longitude values for the given location.

#### Data Variables:
- **GHI:** Global Horizontal Irradiance values (float64) from satellite imagery for each combination of `location_id`, `base_time`, and `step`.
- **solar_elevation:** Solar elevation angles (float32) ranges from 0 to 90, for each combination of `location_id`, `base_time`, and `step`.
- **GHI_CAMS:** GHI from Copernicus Atmosphere Monitoring Service (CAMS) McClear Clear-Sky (float64) for each combination of `location_id`, `base_time`, and `step`.
- **GHI_meas:** Measured Global Horizontal Irradiance values (float64) for each combination of `location_id`, `base_time`, and `step`.

#### Indexes:
There are 3 indexes used to organize the data.

#### Attributes:
- **title:** Describes the content as "Satellite-based cloud motion vector irradiance forecast".
- **remarks:** Notes that the data represents 15-minute averages and is labeled right (end of interval).

This file structure facilitates the analysis and visualization of solar irradiance data, enabling better understanding and forecasting of solar energy patterns.

## 1. Data Exploration

##### 1.0 From the DataSet extract the Latitude and Longitude values.

In [ ]:
print(ds["lat"].item())
print(ds["lon"].item())

##### 1.0 Exactly how many base_times are there in the dataset? (roughtly 6 months x 30 days x 24 hours x 4)

In [ ]:
len(ds.base_time)

##### 1.1 At step=0 what is the mean, median, and standard deviation, for irradiance (GHI) field over the entire time period?

In [ ]:
print(f"Mean GHI: {ds.isel(step=0)['GHI'].mean().item():.2f}")
print(f"Median GHI: {ds.isel(step=0)['GHI'].median().item():.2f}")
print(f"Standard Deviation GHI: {ds.isel(step=0)['GHI'].std().item():.2f}")

##### 1.2 At step=0 what is the mean, median, and standard deviation, for the measured irradiance (GHI_meas) field over the entire time period?

In [ ]:
print(f"Mean GHI: {ds.isel(step=0)['GHI_meas'].mean().item():.2f}")
print(f"Median GHI: {ds.isel(step=0)['GHI_meas'].median().item():.2f}")
print(f"Standard Deviation GHI: {ds.isel(step=0)['GHI_meas'].std().item():.2f}")

##### 1.3 At step=0 what is the mean, median, and standard deviation, for the clearsky irradiance (GHI_CAMS) field over the entire time period?

In [ ]:
print(f"Mean GHI: {ds.isel(step=0)['GHI_CAMS'].mean().item():.2f}")
print(f"Median GHI: {ds.isel(step=0)['GHI_CAMS'].median().item():.2f}")
print(f"Standard Deviation GHI: {ds.isel(step=0)['GHI_CAMS'].std().item():.2f}")

## 2. Solar Zenith

##### 2.0 Create a new variable in the DataSet called the "cos_solar_zenith" $\cos{\theta_z}$, which is calculated as $\cos{(90 ^\circ - \alpha)}$, where $\alpha$ is the  solar elevation. Remeber to first convert degrees to radians.

hint: you can use np.cos, and np.deg2rad

In [11]:
# cos_solar_zenith 0-1
ds["cos_solar_zenith"] = np.cos(np.deg2rad(90 - ds["solar_elevation"]))

## 3. Filter Dataset

##### 3.1 Plot a histogram (with 100 bins) of the GHI values at step=0 .

In [ ]:
plt.hist(ds.isel(step=0)['GHI'].values.flatten(), bins=50)

### 3.2 Create a new dataset called "df_filter" that filters the data based on these conditions:
    1. solar zenith angle > 0 (values during the day)
    2. solar zenith angle at step 0 is greater than 0.15 (forecasting with sufficient illumination of first step)
    3. keep only values where "GHI_meas" and "GHI" are notnull (for evaluation both are required)  

In [13]:
ds_filter = ds.copy()
ds_filter = ds_filter.where(ds_filter["cos_solar_zenith"] > 0)
ds_filter = ds_filter.where(ds_filter["cos_solar_zenith"].isel(step=0) > 0.15)
ds_filter = ds.where(ds_filter['GHI_meas'].notnull() & ds_filter['GHI'].notnull())

In [ ]:
ds_filter

##### 3.2 Plot a histogram of the filtered datset with GHI values at step=0 with 100 bins

In [ ]:
plt.hist(ds_filter.isel(step=0)['GHI'].values.flatten(), bins=50)

##### 3.3 Create boxplots with the filtered data for each week in the dataset plot the GHI values.

In [ ]:
def plot_monthly_ghi_boxplots(ds, step=0, location_id=0):
    """
    Plots boxplots of GHI values for each month.

    Parameters:
    ds (xarray.Dataset): The dataset containing GHI variables.
    step (int, optional): The step index to select. Default is 0.
    location_id (int, optional): The location_id to select. Default is 0.
    """
    # Filter the dataset based on step and location_id
    ds = ds.isel(step=step, location_id=location_id)

    # Group by each month in base_time
    monthly_grouped = ds.groupby('base_time.week')

    # Create a dictionary to store DataFrames for each month
    monthly_dfs = {}

    # Iterate over the grouped object and extract GHI values
    for month, group in monthly_grouped:
        monthly_ghi = group['GHI'].to_dataframe().reset_index()
        monthly_dfs[month] = monthly_ghi[['base_time', 'GHI']].set_index('base_time').rename(columns={'GHI': f'GHI_{month}'})

    # Concatenate all monthly DataFrames into a single DataFrame
    result_df = pd.concat(monthly_dfs.values(), axis=1)

    # Plot boxplots
    plt.figure(figsize=(12, 6))
    result_df.boxplot(grid=False)
    plt.title('Monthly GHI Values')
    plt.xlabel('Month')
    plt.ylabel('GHI')
    plt.xticks(rotation=45)
    plt.show()

plot_monthly_ghi_boxplots(ds_filter)

## Validation

##### 3.3 Create a function that takes an xarray DataSet "ds", a "step" value, and plots a scatter plot, for a given step with the "GHI" on the y-axis, and the "GHI_meas" on the x-axis, to investigate any systematic difference between the observation, and predicted.

In [17]:
def plot_irradiance_comparison(ds, step, max_irr=1200):
    """
    Plots the comparison between measured GHI and satellite forecasted GHI.

    Parameters:
    ds (xarray.Dataset): The dataset containing GHI and GHI_meas variables.
    step (int): The step index to select.
    max_irr (int, optional): The maximum irradiance value for the plot. Default is 1200.
    """
    fig, ax = plt.subplots(figsize=(10, 10))

    ds_ = ds.isel(step=step)
    GHI_step, GHI_meas_step = ds_["GHI"].values.squeeze(), ds_["GHI_meas"].values.squeeze()
    mask = ~np.isnan(GHI_meas_step)

    scatter = ax.scatter(GHI_meas_step[mask], GHI_step[mask], s=2, alpha=0.4, color="blue")
    ax.plot([0, max_irr], [0, max_irr], 'r--')
    ax.set_xlim(0, max_irr)
    ax.set_ylim(0, max_irr)
    ax.set_xlabel('GHI-meas [W/m²]')
    ax.set_ylabel('GHI-sat-forecast [W/m²]')
    ax.title.set_text(f'Irradiance Meas vs Sat Forecast:\nstep={step}, mins={step*30}')

    plt.show()

In [ ]:
plot_irradiance_comparison(ds_filter, step=1)

##### 3.4 Now calculate the root mean squared error between the measured "GHI_meas" and the predicted "GHI" at each step in the dataset. Create a function called "calculate_rmse_at_step" that takes a dataset "ds" and a "step".

The RMSE is given the RMSE formula:  $\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$

hint: Take care to remove nan values

In [19]:
def calculate_rmse_at_step(ds, step):
    ds_ = ds.isel(step=step)
    GHI_step, GHI_meas_step = ds_["GHI"].values.squeeze(), ds_["GHI_meas"].values.squeeze()
    mask = ~np.isnan(GHI_meas_step)

    return np.sqrt(np.nanmean((GHI_step - GHI_meas_step) ** 2))

##### 3.5 Using the function "calculate_rmse_at_step", for all steps in the DataSet, calculate the RMSE value, and plot the step on the "x-axis" agains the RMSE on the "y-axis".

In [ ]:
pd.DataFrame([calculate_rmse_at_step(ds_filter, x) for x in range(13)], columns=["rel_rmse"]).plot()

##### 3.6 Now create a function "rmse"rel" to calculate the RMSE relative to the "GHI_meas for each step, to do this take the RMSE value at each step and divide by the mean of all the "GHI_meas" values for that step.

In [21]:
def rmse_rel(ds, step):
    step_mean_GHI = ds.isel(step=step)["GHI_meas"].mean()
    step_rmse = calculate_rmse_at_step(ds, step)
    return float( step_rmse / step_mean_GHI * 100)

##### 3.5 Using the function "rmse_rel", for all steps in the DataSet, calculate the relative RMSE value, and plot the step on the "x-axis" agains the RMSE on the "y-axis".

In [ ]:
pd.DataFrame([rmse_rel(ds_filter, x) for x in range(13)], columns=["rel_rmse"]).plot()

In [ ]:
!pip install jupyter_contrib_nbextensions

In [ ]:
!pip install jupyter_nbextensions_configurator

In [ ]:
!jupyter contrib nbextension install --user

In [ ]:
!jupyter nbextensions_configurator enable --user